### Imports

In [16]:
import os
import numpy as np
import pandas as pd

### Inputs

In [17]:
fusion_output_df = pd.read_csv("../outputs/fusion_output.csv")
fusion_output_class = fusion_output_df['class'].values
fusion_output_confidence = fusion_output_df['confidence'].values

In [18]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

### Kalman Filter

In [19]:
def kalman_filter_1d(z, Q=1e-4, R=1e-2):
    n = len(z)
    xhat = np.zeros(n)
    P = np.zeros(n)
    xhat[0] = z[0]
    P[0] = 1.0
    for k in range(1, n):
        # previsão
        xhatminus = xhat[k-1]
        Pminus = P[k-1] + Q
        # atualização
        K = Pminus / (Pminus + R)
        xhat[k] = xhatminus + K * (z[k] - xhatminus)
        P[k] = (1 - K) * Pminus
    return xhat

### Apply

In [20]:
conf = kalman_filter_1d(fusion_output_confidence)
cls = np.where(conf < 0.5, 1 - fusion_output_class, fusion_output_class)

In [21]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[cls[i]]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-83    8.3   100    0.72   0        Ruim     0.80      
-89    5.0   58     0.84   0        Ruim     0.80      
-69    5.7   33     0.57   0        Ruim     0.85      
-85    7.5   62     0.81   0        Ruim     0.85      
-88    4.0   36     0.81   0        Ruim     0.86      
-70    4.5   67     0.89   0        Ruim     0.88      
-84    9.3   42     0.73   0        Ruim     0.89      
-70    9.8   98     0.63   0        Ruim     0.87      
-65    9.8   42     0.43   0        Ruim     0.88      
-64    5.4   98     0.90   0        Ruim     0.89      
-74    7.2   40     0.91   0        Ruim     0.89      
-73    7.2   40     0.37   0        Ruim     0.89      
-71    9.0   74     0.58   0        Ruim     0.90      
-82    8.4   90     0.96   0        Ruim     0.89      
-61    6.2   49     0.44   0        Ruim     0.89      
-87    8.3   83     0.70   0        Ruim     0.9

### Save

In [22]:
kalman_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
kalman_output.to_csv("../outputs/kalman_output.csv", index=False)

print("Filtro de Kalman salvo em ../outputs/kalman_output.csv")

Filtro de Kalman salvo em ../outputs/kalman_output.csv
